In [1]:
import outlines
import outlines.models as models
import outlines.text as text

import torch
import transformers

from pydantic import BaseModel, Field, constr, conlist
from enum import Enum

In [2]:
# model = models.awq("TheBloke/Mistral-7B-OpenOrca-AWQ")
model = models.awq("TheBloke/OpenHermes-2.5-Mistral-7B-AWQ")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Replacing layers...: 100%|██████████████████████████████████████████████| 32/32 [00:02<00:00, 13.26it/s]


  0%|          | 0/739 [00:00<?, ?w/s]

Fusing layers...: 100%|█████████████████████████████████████████████████| 32/32 [00:04<00:00,  7.61it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# config = transformers.AutoConfig.from_pretrained(
#     "mosaicml/mpt-7b-8k-instruct", trust_remote_code=True
# )
# config.init_device = "meta"
# model = models.transformers(
#     model_name="mosaicml/mpt-7b-8k-instruct",
#     device="cuda",
#     model_kwargs={
#         "config": config,
#         "trust_remote_code": True,
#         "torch_dtype": torch.bfloat16,
#         "device_map": {"": 0},
#         "cache_dir": "/n/holystore01/LABS/iaifi_lab/Users/smsharma/hf_cache/"
#     },
# )

In [4]:
# config = transformers.AutoConfig.from_pretrained(
#     "teknium/OpenHermes-2.5-Mistral-7B", trusmt_remote_code=True
# )
# config.init_device = "meta"
# model = models.transformers(
    
#     model_name="teknium/OpenHermes-2.5-Mistral-7B",
#     device="cuda",
#     model_kwargs={
#         "config": config,
#         "trust_remote_code": True,
#         "torch_dtype": torch.bfloat16,
#         "device_map": {"": 0},
#         "cache_dir": "/n/holystore01/LABS/iaifi_lab/Users/smsharma/hf_cache/"
#     },
# )

In [194]:
abstract = """
We propose to observe ultraluminous X-ray sources which are located near
objects bright both in the X-rays and the optical using Chandra and HST. The
presence of these reference objects will allow us to tie the x-ray and optical
references frames and achieve 0.1-0.2 arcsecond relative position accuracy in
searching for optical counterparts to the ultraluminous x-ray sources. This
will be a significant improvement over the accuracy previously obtained for
most ULXs {limited by Chandra's absolute astrometry} and will should permit
identification of individual counterparts.
"""

# abstract = """
# We propose a comprehensive optical, UV, and X-ray investigation of the unique
# galaxy POX 52. POX 52 is a Seyfert 1 galaxy with unprecedented properties: its
# host galaxy appears to be a dwarf elliptical, and its stellar velocity
# dispersion is only 36 km/s. The stellar velocity dispersion and the broad
# emission-line widths both suggest a black hole mass of order 10^5 solar
# masses, placing POX 52 in a region of AGN parameter space that is almost
# completely unexplored at present. We request ACS/HRC imaging to perform a
# definitive measurement of the host galaxy structure; STIS UV and optical
# spectroscopy to study the nonstellar continuum and the structure of the
# broad-line region; and Chandra ACS imaging to detect the X-ray emission from
# the nucleus and investigate its spectral and variability properties. The
# results of this program will give a detailed understanding of the host galaxy
# and accretion properties of one of the very few known black holes in the mass
# range around 10^5 solar masses.
# """

# abstract = """
# The observed optical depths to microlensing of stars in the Galactic bulge are
# difficult to reconcile with our present understanding of Galactic dynamics.
# The main source of uncertainty in those comparisons is now shifting from
# microlensing measurements to the dynamical models of the Galactic bar. We
# propose to constrain the Galactic bar models with proper motion observations
# of Bulge stars that underwent microlensing by determining both the kinematic
# identity of the microlensed sources and the importance of streaming motions.
# The lensed stars are typically farther than randomly selected stars.
# Therefore, our proper motion determinations for 36 targeted MACHO events will
# provide valuable constraints on the dynamics of bulge stars as a function of
# distance. The first epoch data for our proposed events is already available in
# the HST archive so the project can be completed within a single HST cycle. The
# exceptional spatial resolution of HST is essential for completion of the
# project. Constraints on te total mass in the bulge will ultimately lead to
# the determination of the amount of dark matter in inner Galaxy.
# """

In [195]:
class ConstrainedResponseHST(BaseModel):
    objects_and_phenomena: str
    science_use_cases: str

In [202]:
@text.prompt
def prompt_fn(abstract):
     """<|im_start|>system
You are an expert astrophysicist, with broad expertise across observational and theoretical astrophysics.
You are able to extract core information from astrophysical texts.

<|im_start|>user
Abstract: "{{abstract}}"

The above is an abstract for a proposed observation taken by the Hubble Space Telescope.

Your will summarize the nature of eventual observation taken by the Telescope, mentioning the astrophysical objects imaged and possible downstream science use cases.

Follow these instructions exactly:
- You will never mention the Hubble Space Telescope, HST, or the HST archive.
- You will only write text and numbers.
- You will not return empty values.
- You will mention the class (e.g., barred spiral galaxy) and not just the specific instance (e.g., Andromeda).
- You will name every single astrophysical object, phenomenon, and possible science use case mentioned in the abstract.
- You will be descriptive in your answers and list multiple items when appropriate. Do not used dashed or numbered lists, only comma-separated.
- The total length of text should not exceed 80 words.

Answer in JSON. The JSON should be a a dictionary with keys "objects_and_phenomena" and "science_use_cases".
Name every single astrophysical objects, phenomena, and  science use cases mentioned in the abstract.

<|im_start|>assistant
"""

prompt = prompt_fn(abstract)

In [203]:
generator = text.generate.json(model, ConstrainedResponseHST)
sequence = generator(prompt)
sequence

ConstrainedResponseHST(objects_and_phenomena='ultraluminous X-ray sources, objects bright in the X-rays and optical, reference objects, relative position accuracy, optical counterparts, x-ray and optical references frames', science_use_cases='tie x-ray and optical references frames, achieve improved accuracy, identify individual counterparts')

In [166]:
# generator = text.generate.continuation(model)
# sequence = generator(prompt)
# sequence

In [236]:
# @text.prompt
# def chain_of_density(abstract):
#     """Abstract: {{ abstract }}

#     The above is an Abstract for an observation taken by the Hubble Space Telescope.
#     You will generate increasingly concise, entity-dense summaries describing the content of the eventual observation and its science use case.

#     Repeat the following 2 steps 4 times.

#     Step 1. Identify 1-3 informative Entities ("; " delimited) from the Abstract which are missing from the previously generated summary.
#     Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

#     A Missing Entity is:
#     - Relevant: to the Hubble observation, describing an astrophysical object or science case.
#     - Specific: descriptive yet concise (5 words or fewer).
#     - Novel: not in the previous summary.
#     - Faithful: present in the Abstract.
#     - Anywhere: located anywhere in the Abstract.

#     Guidelines:
#     - The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "the observation contains") to reach ~80 words.
#     - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
#     - Make space with fusion, compression, and removal of phrases that do not describe the eventual observation.
#     - The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Abstract.
#     - Missing entities can appear anywhere in the new summary.
#     - Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

#     Remember, use the exact same number of words for each summary.

#     Answer in JSON. The JSON should be a a dictionary with key "summaries" that contains a list (length 4) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
#     """


# class Summary(BaseModel):
#     missing_entities: str
#     denser_summary: str

# class Summaries(BaseModel):
#     summaries: conlist(Summary, max_length=4, min_length=4)


In [237]:
# prompt = chain_of_density(abstract)
# result = text.generate.json(model, Summaries)(prompt)
# result.summaries[-1].denser_summary